Start with the regular import statements, and get our raw data from S3 stored in df format

In [257]:
import numpy as np
import pandas as pd

import boto3
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()
bucket='sagemaker-us-east-1-936165954724/ceo-turnover'
data_key = 'SharePriceData.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)

raw_data = pd.read_csv(data_location)

Very shallow data exploration just to see how our df is structured

In [258]:
print(raw_data.head())

   gvkey iid   datadate                   conm  prccm     sic
0   1003  01  31JAN2010  A.A. IMPORTING CO INC   0.02  5712.0
1   1003  01  28FEB2010  A.A. IMPORTING CO INC   0.02  5712.0
2   1003  01  31MAR2010  A.A. IMPORTING CO INC   0.02  5712.0
3   1003  01  30APR2010  A.A. IMPORTING CO INC   0.02  5712.0
4   1003  01  31MAY2010  A.A. IMPORTING CO INC   0.02  5712.0


In [259]:
print(raw_data.columns)

Index(['gvkey', 'iid', 'datadate', 'conm', 'prccm', 'sic'], dtype='object')


We want to calculate percentage returns, but we need a way to make sure we are only taking the percentage change of monthly closing price for the same company, and over the correct time period.

One way to do this is to order the df such that for any two rows that reference the same company, the time difference between the rows follows a uniform pattern. If we check that two rows reference the same company, then, we can extrapolate the time difference between them and use that to calculate the percentage price change.

To do this, we first change the stored dates into a sortable format, st. earlier date < later date, and sort the df by 'gvkey' (so that all data for a given company is contiguous), and then 'date' (so that company data is chronologically ordered). We also sort first by 'sic', which may make the later step of calculating sector returns easier.

In [260]:
month_str_to_num = {"JAN":1, "FEB":2, "MAR":3, "APR":4, "MAY":5, "JUN":6, "JUL":7,
                   "AUG":8, "SEP":9, "OCT":10, "NOV":11, "DEC":12}

def makeDateSortable(date):
    days = int(date[0:2])
    month = month_str_to_num[date[2:5]]
    year = int(date[5:9])
    
    return days + month * 100 + year * 10000

date_series = raw_data['datadate'].apply(lambda x : makeDateSortable(x))

raw_data['date'] = date_series
raw_data = raw_data.sort_values(by=['sic', 'gvkey', 'date'])
raw_data = raw_data.reset_index(drop=True)

print(raw_data.head())

   gvkey iid   datadate       conm  prccm    sic      date
0   1266  01  31JAN2010  ALICO INC  25.64  100.0  20100131
1   1266  01  28FEB2010  ALICO INC  25.96  100.0  20100228
2   1266  01  31MAR2010  ALICO INC  25.25  100.0  20100331
3   1266  01  30APR2010  ALICO INC  26.68  100.0  20100430
4   1266  01  31MAY2010  ALICO INC  25.40  100.0  20100531


Now that since the data is appropriately ordered, and we have monthly data, two rows representing the same company whose indices are n apart should represent data that is n months apart. 

The next step is to write a function that calculates a percentage change in the 'prccm' (closing monthly price) column between rows separated by a supplied n, where n in the number of months we want to calculate the return over. If the two rows being compared represent different companies, there is an error in date ordering (possible if we have missing date points) or trailing security price is not available, then 0 is returned as a dummy value.

In [261]:
day = lambda date : date % 100
month = lambda date : (date // 100) % 100
year = lambda date : date // 10000

def month_difference(date_1, date_2):
    return ((year(date_1) - year(date_2)) * 12) + (month(date_1) - month(date_2))

def pct_chg(current, historical):
    if historical != 0:
        return (current - historical) / historical
    else:
        return 0
    
def trailing_n_months_return(row, n=12):
    if row < n:
        return 0
    
    trailing_date = raw_data.loc[row - n, 'date']
    current_date = raw_data.loc[row, 'date']
    
    trailing_price = raw_data.loc[row - n, 'prccm']
    current_price = raw_data.loc[row, 'prccm']
    
    trailing_id = raw_data.loc[row - n, 'gvkey']
    current_id = raw_data.loc[row, 'gvkey']
    
    if month_difference(current_date, trailing_date) == n and trailing_id == current_id:
        return pct_chg(current_price, trailing_price)
    else:
        return 0

trailing_12_months_return = lambda x : trailing_n_months_return(x, n=12)
trailing_24_months_return = lambda x : trailing_n_months_return(x, n=24)
trailing_36_months_return = lambda x : trailing_n_months_return(x, n=36)

Since our data is now appropriately ordered, and we have written the appropriate transformation functions, we can map those functions to new columns in our dataframe using df.apply() and the anonymous functions defined above.

Disclaimer: This step takes a while to run

In [262]:
# raw_data = raw_data.head(n=1000)

raw_data['idx'] = raw_data.index
raw_data['12_mo_return'] = raw_data['idx'].apply(trailing_12_months_return)
raw_data['24_mo_return'] = raw_data['idx'].apply(trailing_24_months_return)
raw_data['36_mo_return'] = raw_data['idx'].apply(trailing_36_months_return)


Finally, let's check that the transformation was correctly applied! We should see 12_mo_returns starting from row 12, 24_mo_returns from row 24, and 36_mo_returns from row 36. Rows preceding those should have the dummy value of 0.

In [263]:
preview_data = raw_data.drop(['idx', 'iid', 'date', 'sic', 'gvkey'], axis=1).head(n=40)
print(preview_data)

     datadate       conm  prccm  12_mo_return  24_mo_return  36_mo_return
0   31JAN2010  ALICO INC  25.64      0.000000      0.000000      0.000000
1   28FEB2010  ALICO INC  25.96      0.000000      0.000000      0.000000
2   31MAR2010  ALICO INC  25.25      0.000000      0.000000      0.000000
3   30APR2010  ALICO INC  26.68      0.000000      0.000000      0.000000
4   31MAY2010  ALICO INC  25.40      0.000000      0.000000      0.000000
5   30JUN2010  ALICO INC  22.98      0.000000      0.000000      0.000000
6   31JUL2010  ALICO INC  24.36      0.000000      0.000000      0.000000
7   31AUG2010  ALICO INC  20.60      0.000000      0.000000      0.000000
8   30SEP2010  ALICO INC  23.24      0.000000      0.000000      0.000000
9   31OCT2010  ALICO INC  25.60      0.000000      0.000000      0.000000
10  30NOV2010  ALICO INC  23.15      0.000000      0.000000      0.000000
11  31DEC2010  ALICO INC  23.84      0.000000      0.000000      0.000000
12  31JAN2011  ALICO INC  25.11     -0


Looks good! Let's convert the dataframe back into csv format and upload the pre-processed data to S3.


In [275]:
filename = "ShareDataWithReturns.csv"
filepath = "ceo-turnover/" + filename
bucket = 'sagemaker-us-east-1-936165954724'
raw_data.to_csv(filename)


s3 = boto3.resource('s3')
s3.meta.client.upload_file(filename, bucket, filepath)


Aaand we're done.
